In [25]:
from enum import Enum


class Measures(Enum):
    INCH = "in"
    METRE = "m"
    FEET = "ft"
    MINUTE = "min"
    HOUR = "hr"
    SECOND = "sec"
    
    
CONVERSION_RATES = {
    (Measures.METRE, Measures.INCH): 3.28 * 12,
    (Measures.FEET, Measures.INCH): 12,
    (Measures.METRE, Measures.FEET): 3.28,
    (Measures.HOUR, Measures.MINUTE): 60,
    (Measures.MINUTE, Measures.SECOND): 60,
    (Measures.HOUR, Measures.SECOND): 60 * 60
}


def convert(v, from_: Measures, to_: Measures):
    if (from_, to_) not in CONVERSION_RATES and (to_, from_) not in CONVERSION_RATES:
        return None
    if (from_, to_) in CONVERSION_RATES:
        return CONVERSION_RATES[(from_, to_)] * v
    return 1/CONVERSION_RATES[(to_, from_)] * v

In [29]:
assert convert(2, Measures.METRE, Measures.INCH) == 78.72
assert round(convert(13, Measures.INCH, Measures.METRE), 3) == 0.330
assert convert(13, Measures.INCH, Measures.HOUR) == None

In [20]:
CONVERSION_RATES[(Measures.METRE, Measures.INCH)]

39.36

In [ ]:
FACTS = [
    [Measures.FEET, Measures.INCH, 12],
    [Measures.METRE, Measures.FEET, 3.28],
    [Measures.HOUR, Measures.MINUTE, 60],
    [Measures.MINUTE, Measures.SECOND, 60]
]

R = {}

In [74]:
from typing import List, Dict, Set

FACTS = [
    ("ft", "in", 12),
    ("m", "ft", 3.28),
    ("hr", "min", 60),
    ("min", "sec", 60)
]
class Node:
    
    def __init__(self, unit: str):
        self.unit: str = unit
        self.edges: List['Edge'] = []
        
    def add_edge(self, edge: 'Edge') -> None:
        self.edges.append(edge)
        
    def __str__(self):
        return f"{self.unit} {self.edges}"
    
    def __repr__(self):
        return f"{self.unit} {self.edges}"
    
        
class Edge:
    
    def __init__(self, v: float, to_node: 'Node'):
        self.value: float = v
        self.to_node: 'Node' = to_node
        
    def __str__(self):
        return f"{self.value} - {self.to_node}"
    
    def __repr__(self):
        return f"{self.value} - {self.to_node}"

R: dict[str, Node] = {}

def parse_facts(facts):

    for from_, to_, r in facts:
        
        if from_ not in R:
            R[from_] = Node(from_)
        from_node = R[from_]
        
        if to_ not in R:
            R[to_] = Node(to_)
        to_node = R[to_]
        
        from_node.add_edge(Edge(r, to_node))
        to_node.add_edge(Edge(1/r, from_node))
        
def solve(v, from_node: Node, to_node: Node, visited: Set):
    
    if from_node in visited:
        return None
    
    new_visited = set(list(visited) + [from_node])

    for edge in from_node.edges:
        if edge.to_node == to_node:
            return v * edge.value
        s = solve(v * edge.value, edge.to_node, to_node, new_visited)
        if s:
            return s
    return None
        

def answer(v, from_, to_):
    parse_facts(FACTS)
    if from_ not in R:
        return None
    
    if to_ not in R:
        return None
    
    from_node = R[from_]
    to_node = R[to_]
    return solve(v, from_node, to_node, set())

        

In [75]:
assert answer(1, "m", "ft") == 3.28, answer(1, "m", "ft")
assert answer(2, "m", "in") == 78.72, answer(2, "m", "in")
assert round(answer(13, "in", "m"), 3) == 0.330
assert convert(13, "in", "hr") == None

ft ft [12 - in [0.08333333333333333 - ft [...]], 0.3048780487804878 - m [3.28 - ft [...]]]
in in [0.08333333333333333 - ft [12 - in [...], 0.3048780487804878 - m [3.28 - ft [...]]]]
m m [3.28 - ft [12 - in [0.08333333333333333 - ft [...]], 0.3048780487804878 - m [...]]]
hr hr [60 - min [0.016666666666666666 - hr [...], 60 - sec [0.016666666666666666 - min [...]]]]
min min [0.016666666666666666 - hr [60 - min [...]], 60 - sec [0.016666666666666666 - min [...]]]
sec sec [0.016666666666666666 - min [0.016666666666666666 - hr [60 - min [...]], 60 - sec [...]]]
ft ft [12 - in [0.08333333333333333 - ft [...], 0.08333333333333333 - ft [...]], 0.3048780487804878 - m [3.28 - ft [...], 3.28 - ft [...]], 12 - in [0.08333333333333333 - ft [...], 0.08333333333333333 - ft [...]], 0.3048780487804878 - m [3.28 - ft [...], 3.28 - ft [...]]]
in in [0.08333333333333333 - ft [12 - in [...], 0.3048780487804878 - m [3.28 - ft [...], 3.28 - ft [...]], 12 - in [...], 0.3048780487804878 - m [3.28 - ft [...], 3